# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv file cities.csv created in WeatherPy exercise
csvpath = os.path.join("..", "WeatherPy","output_data","cities.csv")
# Create new dataframe vacation_df
vacation_df = pd.read_csv(csvpath)
# Drop the Unamed: 0 column from the dataframe
vacation_df = vacation_df.drop("Unnamed: 0", axis="columns")
vacation_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Verkh-Chebula,56.0308,87.6218,40.64,85,85,8.46,RU,1619654478
1,Nanortalik,60.1432,-45.2371,35.40,93,78,4.45,GL,1619654230
2,Cabo San Lucas,22.8909,-109.9124,75.00,68,40,10.36,MX,1619654217
3,Saraipali,21.3333,83.0000,81.57,11,63,2.82,IN,1619654479
4,Te Anau,-45.4167,167.7167,53.10,90,100,5.93,NZ,1619654479


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Store lat/Lng coordinates from dataframe in locations
locations = vacation_df[["Lat", "Lng"]]
# Convert humidity values to float
humid = vacation_df["Humidity"].astype(float)
# Plot the heatmap
fig = gmaps.figure()
# Configure heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humid, dissipating=False,max_intensity = 100, point_radius=5)
                               
# Add heatmap to figure
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Set conditions for list of cities with ideal weather
ideal_temp = vacation_df.loc[(vacation_df["Max Temp"] < 70)&(vacation_df["Max Temp"] > 50),:]
ideal_humid = ideal_temp.loc[(ideal_temp["Humidity"] > 40) & (ideal_temp["Humidity"]<50),:]
ideal_clouds = ideal_humid.loc[ideal_humid["Cloudiness"] < 45,:]
ideal_clouds.dropna()
ideal_clouds

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,Yulara,-25.2406,130.9889,59.00,44,0,10.36,AU,1619654551
160,Thunder Bay,48.4001,-89.3168,55.40,44,20,8.05,CA,1619654668
305,Eldorado,-23.7869,-54.2836,58.10,46,27,5.46,BR,1619654920
334,Alice Springs,-23.7000,133.8833,57.20,47,0,4.61,AU,1619654689
335,Sanandij,35.3144,46.9923,53.71,46,0,2.80,IR,1619654928
381,Broken Hill,-31.9500,141.4333,57.20,47,0,5.75,AU,1619655001
396,Christchurch,-43.5333,172.6333,64.00,44,7,4.61,NZ,1619654797
489,Chardara,41.2547,67.9692,68.88,47,0,3.47,KZ,1619655152


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Saving to new dataframe named hotels_df
hotel_df = ideal_clouds.copy()
hotel_df = hotel_df.reset_index(drop=True)
# Adding new blank column to hold hotel names
hotel_df["Hotel Name"] = ""

# Set parameters
hotel_radius = 5000
search_type = "lodging"
params = {"type": search_type, "radius": hotel_radius, "key": g_key}

# Start loop of rows in hotel_df to get nearest hotels based on lat/Long
for index, row in hotel_df.iterrows():
    # Establish base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Set the latitude and longitude for parameters
    lat_coord = row["Lat"]
    lon_coord = row["Lng"]
    params["location"] = f"{lat_coord},{lon_coord}"
    # Requests hotel data from google places 
    response = requests.get(base_url, params=params).json()
    hotel_results = response["results"]
    # Populate hotel name column with hotels
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_results[0]["name"]
    
    except:
        print("Missing hotel data....skipping")
        pass
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Yulara,-25.2406,130.9889,59.00,44,0,10.36,AU,1619654551,Desert Gardens Hotel - Ayers Rock Resort
1,Thunder Bay,48.4001,-89.3168,55.40,44,20,8.05,CA,1619654668,Victoria Inn Hotel and Convention Centre
2,Eldorado,-23.7869,-54.2836,58.10,46,27,5.46,BR,1619654920,Villa Verde Hotel
3,Alice Springs,-23.7000,133.8833,57.20,47,0,4.61,AU,1619654689,Desert Palms Alice Springs
4,Sanandij,35.3144,46.9923,53.71,46,0,2.80,IR,1619654928,Sanandaj Tourist Hotel
5,Broken Hill,-31.9500,141.4333,57.20,47,0,5.75,AU,1619655001,Royal Exchange Hotel
6,Christchurch,-43.5333,172.6333,64.00,44,7,4.61,NZ,1619654797,The George
7,Chardara,41.2547,67.9692,68.88,47,0,3.47,KZ,1619655152,Жанаргул


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Configure gmaps
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
# Configure heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humid, dissipating=False,max_intensity = 100, point_radius=5)                            
# Add heatmap to figure
fig.add_layer(heat_layer)
#Add marker layer over heatmap
markers = gmaps.marker_layer(hotel_locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))